# Estratégia de utilização de Bag Of N-gram com Rede Neural

## Imports e definição de funções

In [1]:
import os
import shutil
import time
import pickle
import pandas as pd
import statistics as stat
import numpy as np
import tensorflow as tf
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_1d, global_max_pool
from tflearn.layers.merge_ops import merge
from tflearn.layers.estimator import regression
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
# Função para salvar uma lista em um binário
def save_list_to_file(list_to_save,file_name):
    with open(file_name, 'wb') as f:
        pickle.dump(list_to_save, f)
# Função para carregar uma lista 
def load_list_from_file(file_name):
    with open(file_name, 'rb') as f:
        return pickle.load(f)     
# Criando a classe de parada antecipada
class EarlyStoppingCallback(tflearn.callbacks.Callback):
    def __init__(self, val_epoch_thresh, val_acc_thresh):
        self.val_epoch_thresh = val_epoch_thresh
        self.val_acc_thresh = val_acc_thresh
    def on_epoch_end(self, training_state):
        print("Epoch ", training_state.epoch, " with Accuracy ", training_state.acc_value)
        if training_state.epoch >= self.val_epoch_thresh and training_state.acc_value >= self.val_acc_thresh:
            raise StopIteration
    def on_train_end(self, training_state):
        print("Successfully left training! Final model accuracy:", training_state.acc_value)
print ('Importações e criação de funções realizadas com sucesso')

Importações e criação de funções realizadas com sucesso


## Restaurando o Pickle contendo o DataFrame trabalhado

In [2]:
# Restaurando o DataFrame gerado no Passo 1 - Preparação
FILENAME = 'dataframe.pickle'
df = pd.read_pickle(FILENAME)
df.head()

,classificacao_grupo,classificacao,grupo,descricao_demanda2,solucao,solucao_usuario2,solucao_script2,descricao_demanda,solucao_usuario,solucao_script
0,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - UNACDF...,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,UNACDFPUCOMEX,bom dia ter dois processo voar nao conseguir c...,,prezar usuario carga estar american airlines t...,,"Bom Dia Tenho dois processos o qual ja voo , n...",Prezado Usuário\n\nAs cargas estão com a Améri...,#N/DISP
1,CCT - CONTROLE DE CARGA E TRÂNSITO - UNACDFPUC...,CCT - CONTROLE DE CARGA E TRÂNSITO,UNACDFPUCOMEX,bom dia enviar umar noto parir siscomex pesar ...,2016SOL/00396862,prezadoa usuarioa problema resolver favor test...,usuarioa conformar contato telefonico parir pr...,"Bom dia, Enviei uma nota para o Siscomex e seu...","\n\nPrezado(a) usuário(a),\nProblema resolvido...","Sr.(a) usuário(a), conforme contato telefônic..."
2,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - CAGSBR...,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,CAGSBRPUCOMEX,,2016SOL/00399811,prezar usuariosegue informacao desenvolvimento...,usuarioaconforme informar atraves contato tele...,,"Prezado usuario,segue a informação do nosso de...","Sr.(a) usuário(a),conforme informado através d..."
3,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - UNACDF...,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,UNACDFPUCOMEX,fazer varios tentativo parir obter informacao ...,2017SOL/00003655,prezar usuario voce poder estar tirar suar duv...,usuarioasua solicitacao ser iosamentecentral s...,"Srs., Fizemos várias tentativas para obter inf...","Prezado Usuário,\n\nVocê pode estar tirando as...","\nSr(a). usuário(a),Sua solicitação foi atend..."
4,CCT - CONTROLE DE CARGA E TRÂNSITO - UNACDFPUC...,CCT - CONTROLE DE CARGA E TRÂNSITO,UNACDFPUCOMEX,nro solicitacao comer solicitar nao conseguir ...,2017SOL/0000002832,caro usuario due encontrar averbar,usuarioae necessario parir analise situacao ac...,Nro da Solicitação: 2017SS/0001012491: Como so...,"Caro usuário,\n\nA DUE 17BR0000079935 já se en...","Sr(a). Usuário(a),É necessário, para análise d..."


In [3]:
# Descrevendo os dados do DataFrame
df.describe()

,classificacao_grupo,classificacao,grupo,descricao_demanda2,solucao,solucao_usuario2,solucao_script2,descricao_demanda,solucao_usuario,solucao_script
count,18160,18160,18160,18160,18160,18160,18160,18160,18160,18160
unique,103,21,33,16411,174,8009,124,17059,9208,149
top,PORTAL ÚNICO DE COMÉRCIO EXTERIOR - ATGSMG,DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO,ATGSMG,,,,,,,#N/DISP
freq,3559,6902,4654,490,6564,2240,6739,473,2118,6737


## Criação do Bag Of N-gram para X

In [16]:
# Se essa linha der erro, reinicie o Jupyter com o comando abaixo:
#
#  jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10
#
start_time = time.time()
MAX_FEATURES=20000
# Cria um ngram vectorizer
# Vamos criar o BOW dos 2 documentos
# Usamos n_gram = 2
#vectorizer = CountVectorizer(ngram_range=(2,2), max_features=MAX_FEATURES)
vectorizer = TfidfVectorizer(ngram_range=(2,2),stop_words=stopwords.words('portuguese'),strip_accents='unicode',max_features=MAX_FEATURES)
# Contabilizamos as combinações de caracteres nas palavras
X_vet = vectorizer.fit_transform(df['descricao_demanda2'])
print('Criação de X em %.2f segundos' % (time.time() - start_time))
print('Formato de X_vet: ',X_vet.shape)

Criação de X em 1.25 segundos
Formato de X_vet:  (18160, 20000)


In [17]:
# Imprimindo os valores para cada token no primeiro documento
for doc in X_vet:
    print('------------')
    scores = zip(vectorizer.get_feature_names(),
                 np.asarray(doc.sum(axis=0)).ravel())
    sorted_scores = sorted(scores, key=lambda x: x[1], reverse=True)
    for item in sorted_scores:
        if item[1] > 0.0:
            print ("{0:30} Score: {1}".format(item[0], item[1]))
    break

------------
dues respectivo                Score: 0.3161551209918496
sair estocar                   Score: 0.3045893329234719
datar horario                  Score: 0.29999936792939363
dois processo                  Score: 0.2959507318751046
processed listener             Score: 0.2789149279685556
efetuar manifestacao           Score: 0.27408945926724537
manifestacao ser               Score: 0.2534386524829566
dar ser                        Score: 0.2398729366579298
ter dois                       Score: 0.23913368930058923
concluir manifestacao          Score: 0.2356586440070707
nao concluir                   Score: 0.21757488972029793
dia ter                        Score: 0.21387522800613093
conseguir concluir             Score: 0.21085511011858324
manifestacao dar               Score: 0.17406928228528926
ser informar                   Score: 0.17384964142768913
listener attachment            Score: 0.1487872207995438
nao conseguir                  Score: 0.11273795474123287
bom dia  

## Criação do One Hot Encoding para X

In [18]:
# Preparando dados de X
start_time = time.time()
X_vocabulary = vectorizer.get_feature_names()
NUM_FEATURES = len(X_vocabulary)
print('Tamanho do vocabulário: {}'.format(len(X_vocabulary)))
print('Imprimindo os 100 primeiros:')
print(X_vocabulary[:100])
print('--------------------------------------------------------------------------------------------------------------')
X_values = X_vet.toarray().astype(float)
print('Formato de X {}:'.format(X_values.shape))
print('Imprimindo One-Hot Encoding dos 5 primeiros:')
print(X_values[:5])
print('Criação de Y em %.2f segundos' % (time.time() - start_time))

Tamanho do vocabulário: 20000
Imprimindo os 100 primeiros:
['abaixar agente', 'abaixar aguardar', 'abaixar algum', 'abaixar anexar', 'abaixar aparecer', 'abaixar apos', 'abaixar apresentar', 'abaixar arquivar', 'abaixar atenciosamente', 'abaixar att', 'abaixar campar', 'abaixar carga', 'abaixar comer', 'abaixar conformar', 'abaixar constar', 'abaixar consulto', 'abaixar cpf', 'abaixar dar', 'abaixar descricao', 'abaixar desde', 'abaixar destacar', 'abaixar due', 'abaixar dues', 'abaixar duexer', 'abaixar entrar', 'abaixar enviar', 'abaixar errar', 'abaixar estar', 'abaixar etapalendo', 'abaixar exemplo', 'abaixar exportador', 'abaixar favor', 'abaixar fazer', 'abaixar ficar', 'abaixar fiscal', 'abaixar gentileza', 'abaixar grato', 'abaixar image', 'abaixar imagem', 'abaixar informacao', 'abaixar informar', 'abaixar item', 'abaixar lpco', 'abaixar mencionar', 'abaixar mensagem', 'abaixar momento', 'abaixar msg', 'abaixar nao', 'abaixar notar', 'abaixar noto', 'abaixar numero', 'abaixar 

## Criação do One Hot Encoding para Y

In [19]:
# Preparando dados de Y
#Vetorização dos labels (Target)
Y_labels = pd.get_dummies(df['classificacao'])
print('Quantidade de targets: {}'.format(len(Y_labels)))
print('Imprimindo os targets:')
print('Targets: ',Y_labels.columns)
print('--------------------------------------------------------------------------------------------------------------')
Y_values = pd.get_dummies(df['classificacao']).values
print('Formato de Y {}:'.format(Y_values.shape))
print('Imprimindo One-Hot Encoding do primeiro Y:')
print(Y_values[0])

Quantidade de targets: 18160
Imprimindo os targets:
Targets:  Index(['AMBIENTE', 'ANÁLISE URC', 'APLICAÇÃO',
       'CA - CONFERÊNCIA ADUANEIRA EXPORTAÇÃO',
       'CADASTRO DE INTERVENIENTES - RFB',
       'CCT - CONTROLE DE CARGA E TRÂNSITO', 'DA - DESPACHO ADUANEIRO',
       'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO',
       'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - MDIC',
       'DUEX - DECLARAÇÃO ÚNICA DE EXPORTAÇÃO - RFB', 'DÚVIDA',
       'DÚVIDA OU INFORMAÇÕES', 'GR – GERENCIAMENTO DE RISCO ALEATÓRIOS',
       'OPERADOR ECONÔMICO AUTORIZADO (OEA) - RFB',
       'PORTAL ÚNICO DE COMÉRCIO EXTERIOR',
       'PORTAL ÚNICO SISCOMEX ANUENTES INTERNET (PUCOMEX)',
       'PORTAL ÚNICO SISCOMEX RFB INTRANET(PUCOMEX)',
       'PROSPECÇÃO - CLASSIF - CLASSIFICAÇÃO FISCAL DE MERCADORIAS RFB',
       'SITUAÇÃO NÃO PREVISTA', 'TA-LPCO - TRATAMENTO ADMIN',
       'VICOMEX - VISÃO INTEGRADA DO COMÉRCIO EXTERIOR - MDIC'],
      dtype='object')
-------------------------------------------------------

## Separação dos dados de treinamento e teste

In [20]:
# Separando dados de treinamento e testes
if 'X_values' in locals():
    X_train, X_test, y_train, y_test = train_test_split(X_values, Y_values, test_size=0.33, random_state=42)
    print(' Train data: X_train ',X_train.shape,', y_train',y_train.shape)
    print(' Test data: X_test ',X_test.shape,', y_test ',y_test.shape)
    print('Exemplo de X_train: ',X_train[0][:100],'...')
    print('Exemplo de y_train: ',y_train[0])
    del X_values, Y_values

 Train data: X_train  (12167, 20000) , y_train (12167, 21)
 Test data: X_test  (5993, 20000) , y_test  (5993, 21)
Exemplo de X_train:  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.] ...
Exemplo de y_train:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


## Criação de uma Rede Neural DNN para ser treinada

In [21]:
# Imprimindo primeiro dado de treino
print('Entrada no formato (indice:valor)')
i = 0
for x in X_train[0]:
    if x != 0.0:
        print (i,":",x)
    i += 1
print('Resultado esperado: ',y_train[0])   

Entrada no formato (indice:valor)
286 : 0.3023616380474234
1820 : 0.31948917354902495
3141 : 0.2914867102735966
7138 : 0.23506512297941448
8132 : 0.29243335715329116
9523 : 0.3017668707214662
11911 : 0.29717893474093193
12139 : 0.2329235157226456
13337 : 0.3025609988309158
14249 : 0.2912306952134304
16703 : 0.29526075355654563
18124 : 0.2891321652888649
Resultado esperado:  [0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]


In [22]:
DATASET = 'pucomex'
REDE = 'net_3'
BATCH = 32
EPOCHS = 100
MIN_EPOCHS = 5
MIN_ACCURACY = 0.99
print ('Parâmetros setados')

Parâmetros setados


In [23]:
# Reset do grafo
tf.reset_default_graph()
# Cria a rede neural
net = None
if REDE == 'net1':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_1':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_2':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_3':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_3_1':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.4)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.4)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_3_2':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.3)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.3)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_4':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_5':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_6':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 128)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 64)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, 32)
    net = tflearn.dropout(net, 0.5)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net_cnn_1':
    net = input_data(shape=[None, X_train.shape[1]], name='input')
    net = tflearn.embedding(net, input_dim=NUM_FEATURES, output_dim=128)
    branch1 = conv_1d(net, 32, 3, padding='same', activation='relu', regularizer="L2")
    branch2 = conv_1d(net, 32, 5, padding='same', activation='relu', regularizer="L2")
    net = merge([branch1, branch2], mode='concat', axis=1)
    net = tf.expand_dims(net, 2)
    net = global_max_pool(net)
    net = dropout(net, 0.5)
    net = fully_connected(net, len(y_train[0]), activation='softmax')
    net = regression(net, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')
elif REDE == 'net_8':
    net = input_data(shape=[None, X_train.shape[1]], name='input')
    net = tflearn.embedding(net, input_dim=NUM_FEATURES, output_dim=128)
    branch1 = conv_1d(net, 128, 3, padding='same', activation='relu', regularizer="L2")
    branch2 = conv_1d(net, 128, 5, padding='same', activation='relu', regularizer="L2")
    branch3 = conv_1d(net, 128, 7, padding='same', activation='relu', regularizer="L2")
    net = merge([branch1, branch2, branch3], mode='concat', axis=1)
    net = tf.expand_dims(net, 2)
    net = global_max_pool(net)
    net = dropout(net, 0.5)
    net = fully_connected(net, len(y_train[0]), activation='softmax')
    net = regression(net, optimizer='adam', learning_rate=0.001,
                     loss='categorical_crossentropy', name='target')
elif REDE == 'net2':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 16)
    net = tflearn.fully_connected(net, 16)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net3':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, 8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net4':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.fully_connected(net, 100)
    net = tflearn.fully_connected(net, 50)
    net = tflearn.fully_connected(net, 25)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net)
elif REDE == 'net5':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.embedding(net, input_dim=len(X_train[0]), output_dim=len(y_train[0]))
    net = tflearn.lstm(net, 8)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net, optimizer='adam',loss='binary_crossentropy')
elif REDE == 'net6':
    net = tflearn.input_data(shape=[None, len(X_train[0])])
    net = tflearn.embedding(net, input_dim=len(X_train[0]), output_dim=len(y_train[0]))
    net = tflearn.lstm(net, 8)
    net = tflearn.dropout(net, 0.8)
    net = tflearn.fully_connected(net, len(y_train[0]), activation='softmax')
    net = tflearn.regression(net, optimizer='adam',loss='binary_crossentropy')
# Define o modelo e configura o tensorboard
print ('Criada a',REDE)
model = tflearn.DNN(net, tensorboard_verbose=3, tensorboard_dir='/home/03662232677/c4/PUCOMEX/tflearn_logs_'+str(DATASET)+'/'+str(DATASET)+'_n'+str(NUM_FEATURES)+'_st'+str(EPOCHS)+'_'+str(REDE)+'_b'+str(BATCH))

Criada a net_3


## Treinamento da Rede Neural com os dados de treino

In [24]:
# Iniciando o treinamento da rede neural. Acompanhe o treinamento pelo TensorBoard utilizando o comando abaixo via terminal:
#
# tensorboard --logdir tflearn_logs_...
# 
start_time = time.time()
early_stopping_cb = EarlyStoppingCallback(val_epoch_thresh=MIN_EPOCHS,val_acc_thresh=MIN_ACCURACY)
# Treinamento
try:
    model.fit(X_train, y_train, n_epoch = EPOCHS, batch_size = BATCH, show_metric = True, callbacks=early_stopping_cb)
except StopIteration:
    print("Caught callback exception. Returning control to user program.")
print('Finalização do treinamento em %.2f segundos' % (time.time() - start_time))

Training Step: 38099  | total loss: 0.22486 | time: 10.443s
| Adam | epoch: 100 | loss: 0.22486 - acc: 0.9228 -- iter: 12160/12167
Training Step: 38100  | total loss: 0.26372 | time: 10.469s
| Adam | epoch: 100 | loss: 0.26372 - acc: 0.9086 -- iter: 12167/12167
--
Epoch  100  with Accuracy  0.9086063504219055
Successfully left training! Final model accuracy: 0.9086063504219055
Finalização do treinamento em 1081.06 segundos


## Calculando a acurácia

In [25]:
# Testando o modelo com a base de testes
total = 0
acertos = 0
score1 = model.evaluate(X_test, y_test)
print('Acurácia para os dados de teste: {}'.format(score1))
score2 = model.evaluate(X_train, y_train)
print('Acurácia para os dados de treinamento: {}'.format(score2))
print('Acurácia para todos os dados: {}'.format(stat.mean(score1+score2)))

Acurácia para os dados de teste: [0.5344568668413536]
Acurácia para os dados de treinamento: [0.9413166762652428]
Acurácia para todos os dados: 0.7378867715532982


## Realizando um teste de um documento

In [26]:
N = 3
#print(X_test[N])
print(np.rint(model.predict([X_test[N]]))[0])
print(y_test[N])

[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


## Salvando um modelo treinado

In [15]:
# Save model and pickle files
model_name = 'model_'+str(DATASET)+'_n'+str(MAX_FEATURES)+'_st'+str(EPOCHS)+'_'+str(REDE)+'_b'+str(BATCH)
directory = '/home/03662232677/c4/PUCOMEX/'+model_name
if not os.path.exists(directory):
    os.mkdir(directory) 
model.save(directory+'/'+model_name+'.tflearn')
save_list_to_file(X_vocabulary,directory+'/X_vocabulary.pickle')
#save_list_to_file(X_values,directory+'/X_values.pickle')
#save_list_to_file(Y_labels,directory+'/Y_labels.pickle')
#save_list_to_file(Y_values,directory+'/Y_values.pickle')
save_list_to_file(X_train,directory+'/X_train.pickle')
save_list_to_file(X_test,directory+'/X_test.pickle')
save_list_to_file(y_train,directory+'/y_train.pickle')
save_list_to_file(y_test,directory+'/y_test.pickle')

INFO:tensorflow:/home/03662232677/c4/PUCOMEX/model_pucomex_n10000_st100_net_3_b32/model_pucomex_n10000_st100_net_3_b32.tflearn is not in all_model_checkpoint_paths. Manually adding it.


FIM